In [3]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from GptCOTPrompter_BeamSeach import *
from AutoReasoner import *
import dotenv

config = dotenv.dotenv_values(".env")
openai.api_type = 'azure'
openai.api_base = 'https://meta-prompter-az-openai.openai.azure.com'
openai.api_version = '2022-12-01'
openai.api_key = config['OPENAI_API_KEY_ms']

# print(openai.Model.list())
dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables-sample400.tsv', sep='\t')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')
ft = None

In [2]:
# import fasttext
# ft = fasttext.load_model('cc.en.300.bin')

In [5]:
# import sys
# print(sys.executable)

/usr/bin/python3


In [3]:
NNDemo = False
max_demo = 5
# template = 'original-sql-py'
template = 'original-sql-py-no-intermediate'
# template = 'formatv1-sql-py'

# gpt_model = 'text-davinci-003'
gpt_model = 'mp-aoi-codex'
# gpt_model = 'gpt-3.5-turbo'

def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_LeverVote(
                                              f'prompt_template/{template}.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'],  
                                              base_path='./dataset/WikiTableQuestions/',
                                              demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                             )
            codex_prompter.model = gpt_model
            codex_prompter.max_demo = max_demo
            
            codex_prompter._gen_gpt_prompt(NNDemo, ft)
            codex_prompter._get_gpt_prediction(5)
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    return log
    
for program in ['sql-py']:
    n_threads = 1
    maxLimit = float('inf')
    # maxLimit = 1
    from joblib import Parallel, delayed
    output_result_file = f'./dataset/WikiTableQuestions/results/CodexAnswerCOTExecutor_LeverVote_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables_limit{maxLimit}_model{gpt_model}.json'
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(output_result_file, 'w'), indent=4)
    # evaluate: 
    os.system(f'cd ./dataset/WikiTableQuestions/ && python2 evaluator.py ./results/{output_result_file.split("/")[-1]} && cd ..')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.97s/it]
Reading dataset from ./tagged/data/training.tagged
Reading dataset from ./tagged/data/pristine-seen-tables.tagged
Reading dataset from ./tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./results/CodexAnswerCOTExecutor_LeverVote_original-sql-py-no-intermediate_sql-py_NNDemo=False_results_pristine-unseen-tables_limit1_modelmp-aoi-codex.json
Examples: 1
Correct: 0
Accuracy: 0.0
GPT error: 0.0
Uncaught error: 0.0
{}


In [10]:
NNDemo = False
max_demo = 5
template = 'original-sql-py-no-intermediate'
program = 'sql-py'
# gpt_model = 'text-davinci-003'
gpt_model = 'mp-aoi-codex'

def func(i):
    codex_prompter = CodexAnswerCOTExecutor_LeverVote(
                                      f'prompt_template/{template}.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'],  
                                      base_path='./dataset/WikiTableQuestions/',
                                      demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                     )
    codex_prompter.model = gpt_model
    codex_prompter.max_demo = max_demo
    codex_prompter.demo_ids = [0, 1, 2, 3, 6, 8, 11]


    # codex_prompter._gen_gpt_prompt()
    codex_prompter._gen_gpt_prompt(NNDemo, ft)
    codex_prompter._get_gpt_prediction(5)
    log = codex_prompter._log_dict()
    # print(vars(codex_prompter))
    # print(codex_prompter.frequency_penalty)
    return log
a = func(0)

In [11]:
# print(a['prompt'])
# print(a)
a

{'id': 'nu-0',
 'utterance': 'which country had the most cyclists finish within the top 10?',
 'source_csv': './dataset/WikiTableQuestions/csv/203-csv/733.csv',
 'target_value': 'Italy',
 'predicted_value': 'ESP',
 'prompt': 'The database table DF is shown as follows:\n[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss\n---\n[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18\n[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20\n[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14\n...\n[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108\n[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan\

In [9]:
import requests
print(requests.__version__)
requests.adapters.HTTPAdapter(max_retries=5)

2.27.1
